# Gallery Example: M/H₂/1 Queue (Hyperexponential Service)

This example demonstrates an M/H₂/1 queueing system:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Hyperexponential service times (more variable than exponential)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Hyperexponential service distribution has higher variance than exponential, representing inconsistent service times (SCV=4).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mhyp1(): Network {    """Create M/H₂/1 queueing model"""    model = Network("M/H/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    # Exponential arrivals with rate λ=1    source.setArrival(oclass, Exp(1))    # Hyperexponential service with mean=0.5 and SCV=4    queue.setService(oclass, HyperExp.fit_mean_and_scv(0.5, 4))        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mhyp1()

## Theoretical Analysis for M/H₂/1

For M/H₂/1 with:
- **Arrival rate**: λ = 1 (Exponential)
- **Service time**: Hyperexponential with mean=0.5, SCV=4
- **Utilization**: ρ = λ × E[S] = 1 × 0.5 = 0.5

The Hyperexponential service characteristics:
- **Mean**: 0.5
- **Squared Coefficient of Variation**: C²ₛ = 4
- **Variance**: 4 × (0.5)² = 1.0 (much higher than exponential variance = 0.25)

Higher service variability leads to worse performance than M/M/1.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare service time variability effects
println("\n=== Service Variability Comparison ===")

fun create_service_model(dist_type, scv=None, order=None): Network {
    """Create model with different service distributions"""
    val model_svc = Network(f"M/{dist_type}/1")
    val source = Source(model_svc, "Source")
    val queue = Queue(model_svc, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_svc, "Sink")
    
    val oclass = OpenClass(model_svc, "Class")
    source.setArrival(oclass, Exp(1))
    
    if dist_type == "D":
// Deterministic service
        queue.setService(oclass, Det(0.5))
    elif dist_type == "E":
// Erlang service
        queue.setService(oclass, Erlang.fit_mean_and_order(0.5, order or 2))
    elif dist_type == "M":
// Exponential service
        queue.setService(oclass, Exp(2))
    elif dist_type == "H":
// Hyperexponential service
        queue.setService(oclass, HyperExp.fit_mean_and_scv(0.5, scv or 4))
    
    val P = model_svc.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_svc.link(P)
    
    return model_svc
// Compare different service distributions
val distributions = [
    ("D", 0.0, None, "Deterministic"),
    ("E", 0.5, 2, "Erlang-2"),
    ("M", 1.0, None, "Exponential"),
    ("H", 4.0, None, "Hyperexp")
]

println("Service Type    | C²ₛ | Response Time | Queue Length")
println("-" * 50)

for dist_type, cv_sq, order, name in distributions:
    try:
        if dist_type == "D":
            val model_dist = create_service_model(dist_type)
        elif dist_type == "E":
            val model_dist = create_service_model(dist_type, order=order)
        elif dist_type == "H":
            val model_dist = create_service_model(dist_type, scv=cv_sq)
        else:
            val model_dist = create_service_model(dist_type)
        
        val solver = MVA(model_dist)
        val avg_table = solver.avgTable
        
        val resp_time = float(avg_table.iloc[1, 2])
        val queue_length = float(avg_table.iloc[1, 3])
        
        println(f"{name:12s}    | {cv_sq:.1f} |     {resp_time:.4f}    |    {queue_length:.4f}")
    except Exception as e:
        println(f"{name:12s}    | {cv_sq:.1f} | Error: {str(e)[:20]}...")

println("\nObservation: Performance degrades as service variability (C²ₛ) increases.")

In [ ]:
// Pollaczek-Khinchine formula validation
println("\n=== Pollaczek-Khinchine Formula Validation ===")
// For M/G/1, the mean response time is given by:
// W = E[S] + (λ × E[S²]) / (2 × (1 - ρ))
// where E[S²] = Var[S] + (E[S])²

val lambda_rate = 1.0
val mean_service = 0.5
val rho = lambda_rate * mean_service
val scv = 4.0
val var_service = scv * (mean_service ** 2)
val second_moment = var_service + (mean_service ** 2)
// Theoretical calculation
val w_theory = mean_service + (lambda_rate * second_moment) / (2 * (1 - rho))
val l_theory = lambda_rate * w_theory
// Simulation results
val solver = MVA(model)
val avg_table = solver.avgTable
val w_simulation = float(avg_table.iloc[1, 2])
val l_simulation = float(avg_table.iloc[1, 3])

println(f"M/H₂/1 with λ=1, E[S]=0.5, C²ₛ=4:")
println(f"")
println(f"Response Time:")
println(f"  Theory (P-K):     {w_theory:.4f}")
println(f"  Simulation:       {w_simulation:.4f}")
println(f"  Relative Error:   {abs(w_theory - w_simulation) / w_theory * 100:.2f}%")
println(f"")
println(f"Queue Length:")
println(f"  Theory (Little):  {l_theory:.4f}")
println(f"  Simulation:       {l_simulation:.4f}")
println(f"  Relative Error:   {abs(l_theory - l_simulation) / l_theory * 100:.2f}%")

println(f"\nPollaczek-Khinchine components:")
println(f"  E[S] = {mean_service}")
println(f"  Var[S] = {var_service}")
println(f"  E[S²] = {second_moment}")
println(f"  ρ = {rho}")
println(f"  Waiting time = {w_theory - mean_service:.4f}")
println(f"  Service time = {mean_service}")